In [3]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
imgs = pd.read_csv('isomap.csv', header=None)
imgs.shape

(4096, 698)

In [13]:
X = imgs.values.T

dists = cdist(X, X, 'euclidean')
def get_a_matrix(a):
    idx = a.argsort().argsort()
    idx = np.array([1 if x <=100 and x>0 else 0 for x in idx]).reshape(idx.shape)
    return idx
def get_w_matrix(a):
    idx = a.argsort().argsort()
    idx = np.array([x if x <=100 else np.inf for x in idx]).reshape(idx.shape)
    return idx

A = np.apply_along_axis(get_a_matrix, 1, dists)
W = np.apply_along_axis(get_w_matrix, 1, dists)


In [14]:
import numpy as np
import networkx as nx


def Matrix_D(W):
    # Generate Graph and Obtain Matrix D, \\
    # from weight matrix W defining the weight on the edge between each pair of nodes.
    # Note that you can assign sufficiently large weights to non-existing edges.
    
    # Package networkx (version 1.11) is required

    n = np.shape(W)[0]
    Graph = nx.DiGraph()
    for i in range(n):
        for j in range(n):
            Graph.add_weighted_edges_from([(i,j,min(W[i,j], W[j,i]))])

    res = nx.all_pairs_dijkstra_path_length(Graph)
    D = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            D[i,j] = res[i][j]
    np.savetxt('D.csv', D)
    return D


In [15]:
D = Matrix_D(W)

In [11]:
D

array([[ 0., 22.,  2., ..., 33., 21., 13.],
       [22.,  0., 23., ..., 13., 11., 21.],
       [ 2., 23.,  0., ..., 35., 22., 11.],
       ...,
       [33., 13., 35., ...,  0., 24., 33.],
       [21., 11., 22., ..., 24.,  0., 16.],
       [13., 21., 11., ..., 33., 16.,  0.]])

In [26]:
from sklearn.preprocessing import KernelCenterer
from sklearn.metrics.pairwise import pairwise_kernels
H = KernelCenterer().fit_transform(D)
C = np.dot(np.dot(H,D**2),H)


In [33]:
vals, vecs = np.linalg.eig(C)
pc_vals = vals[vals.argsort()[::-1]]
pcs = vecs[vals.argsort()[::-1]]

In [39]:
np.dot(pcs[:,:2], pc_vals[:2])

array([ 23385381.53428476,  32286300.66812805, -30751986.32023316,
        -1997083.52741386,  40179400.19180119,  28761586.97269851,
        68700397.74518955, -13015911.98667073,  64658851.14037646,
        -4475671.509177  ,  78838754.23470224, -28380563.26982944,
       -44696322.70373678, -37858318.37025183, -12786944.14751597,
       -71622123.33864373,  62009149.73973154, -65608376.63481276,
       -69334012.50557481, -29884878.40198534,  -2108084.73638073,
       -33086325.31551117,  77837473.4631334 , -37801757.78560115,
        65197265.72871371,   2606054.03678793,  17324597.65813773,
       -29883298.26904883, -11326226.30642679, -31427480.60525341,
       -39609070.34592737, -43666241.43732208,  12947455.59322794,
        28511527.3636104 ,  80719941.59547836,   6801906.13796566,
        49277777.65570441, -20152827.68123055, -40910488.22262593,
        63719554.46428261,  69404275.83782601, -11366401.54605436,
       -28779656.31784789, -52038892.18244225,   5224452.97876

array([[ 0.04312657, -0.03583853,  0.01606214, ...,  0.00341179,
        -0.04142125,  0.01099242],
       [ 0.04827968, -0.03015638,  0.02939212, ..., -0.00059016,
        -0.04351708, -0.05679632],
       [-0.04745125,  0.03123853,  0.02520703, ...,  0.02104602,
        -0.03228683,  0.05775126],
       ...,
       [-0.03251797, -0.02123959, -0.02707016, ..., -0.00029875,
        -0.04334296, -0.00585323],
       [ 0.04263098,  0.0136899 , -0.02195766, ..., -0.01074603,
        -0.04879928,  0.01254064],
       [-0.03204981, -0.01498615, -0.02853931, ...,  0.03583441,
        -0.02454738,  0.00274013]])